# **Agentic RAG for Analyzing Customer Issues**
#### Discover how Agentic RAG is transforming customer support by boosting decision-making, increasing efficiency, and offering personalized, proactive solutions with minimal human involvement

# Introduction
Imagine a customer support team overwhelmed by an endless stream of customer queries, struggling to gather insights from various systems and products. The result? Slow, inaccurate responses and frustrated customers. The time spent searching manually and handling repetitive tasks only adds to the inefficiency.

Now, envision a system that effortlessly retrieves relevant information from all these sources, makes intelligent decisions, and delivers personalized solutions, all on its own. With automation at its heart, this system minimizes the need for human intervention, saving time and enhancing both speed and accuracy.

This is the power of Agentic RAG. It transforms customer support into a smoother, faster, and more precise process, ensuring quick resolutions and lightening the load for support teams.

In this blog, we’ll delve into the ways Agentic RAG is revolutionizing customer support systems. From tackling common challenges to showcasing practical applications, we’ll uncover how this technology provides actionable insights for business leaders looking to optimize operations and for AI enthusiasts eager to explore its potential.



# Can’t we do it using traditional RAG? Why do we need Agentic RAG?

Well, traditional RAG models are good at retrieving relevant information and generating responses. However, they often fall short in one key area — autonomy. While they can handle single-step tasks well, they lack the ability to decide the next best step in complex, multi-step problem-solving. Without the ability to adapt and make decisions on their own, traditional RAG systems struggle with issues that require deeper understanding or follow-up actions. This makes them less effective when it comes to addressing evolving or intricate customer problems.

By not having the ability to automatically refine or escalate tasks, traditional RAG models can leave customers without a clear path to resolution, limiting their effectiveness in customer support.

# What is Agentic RAG?
Agentic RAG (Retrieval-Augmented Generation) is an advanced AI system that integrates the power of information retrieval with generative capabilities, all while incorporating autonomous agent-like behavior. This system is designed to go beyond traditional AI models by actively making decisions and adjusting its actions based on task feedback, making it highly adaptive and goal-oriented.

### How It Works:
* Retrieval: The first step in the process involves fetching relevant data from vast knowledge bases or databases. This ensures that the system has access to accurate, up-to-date information to address customer queries or support issues.
* Generation: Once the relevant data is retrieved, the system generates human-like responses or summaries, effectively mimicking natural conversation and providing solutions in an easy-to-understand format.
* Agentic Behavior: Unlike traditional models, Agentic RAG introduces a feedback loop that allows it to continuously refine its actions. If an initial response doesn’t fully resolve the issue or needs further clarification, the system can make autonomous decisions to either escalate the issue or adjust its approach based on the feedback it receives.
Agentic RAG works with little human input and makes independent, smart decisions. It adapts to changing customer needs, constantly improving its methods as situations shift. Focused on clear goals, it delivers efficient and effective solutions, driving results with accuracy.

In [ ]:
# pip install -q llama-index-core
# pip install -q llama-index-readers-file
# pip install -q llama-index-embeddings-openai
# pip install -q llama-index-llms-llama-api
# pip install -q crewai

In [ ]:
import os
import pandas as pd
from crewai import Agent, Task, Crew, Process
from crewai_tools import LlamaIndexTool
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.llms.openai import OpenAI

In [ ]:
# connect your Google Drive
from google.colab import drive
drive.mount('/content/gdrive')


In [ ]:
reader = SimpleDirectoryReader(input_files=["/content/customer_support_tickets.csv"])
docs = reader.load_data()
docs

Dynamic brand filtering plays a vital role in delivering accurate insights. It enables flexible and precise analysis by identifying brands directly within the dataset, removing the reliance on predefined brand lists. This approach simplifies the process and enhances efficiency, making it a powerful tool for data-driven decision-making.

In [ ]:
def get_brands_from_dataset(file_path):
    """Extract unique brands from the dataset."""
    df = pd.read_csv(file_path)  
    unique_brands = df['Brand'].unique().tolist()  
    return unique_brands

In [ ]:
# Load dataset and extract brands
file_path = "/content/customer_support_tickets.csv"  
brands_in_data = get_brands_from_dataset(file_path)
brands_string = "', '".join(brands_in_data)
brands_string

"GoPro', 'LG', 'Dell', 'Microsoft', 'Autodesk', 'Sony', 'Xbox', 'HP', 'Amazon', 'Adobe', 'Samsung', 'iPhone"

In [ ]:
import os
from google.colab import userdata

os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
llm = OpenAI(model="gpt-4o", temperature=0.5)

In [160]:
index = VectorStoreIndex.from_documents(docs)
query_engine = index.as_query_engine(similarity_top_k=5, llm=llm)

In [161]:
query_tool = LlamaIndexTool.from_query_engine(
    query_engine,
    name="Customer Support Query Tool",
    description="Use this tool to lookup the brand data in the data in '{brands_string}' and generate insights only for these brand"
)

In [ ]:
analyst = Agent(
      role="Customer Ticket Analyst",
      goal="Uncover insights about customer issues trends",
      backstory="""You work at a Product Company. Your goal is to understand customer issues patterns for each of the Brand that is part of the dataset.""",
      verbose=True,
      allow_delegation=False,
      tools=[query_tool],
  )


writer = Agent(
      role="Product Content Specialist",
      goal="""Craft compelling content on customer issues trends for each of the brands in the dataset""",
      backstory="""You are a renowned Content Specialist, known for your insightful and engaging articles.
    You transform complex sales data into compelling narratives.""",
      verbose=True,
      allow_delegation=False,
  )



In [ ]:
task1 = Task(
      description = """Analyze the top customer issues for each of the brands in '{brands_string}'.
      Use the columns Ticket Type, Ticket Subject, and Ticket Description to generate insights.""",
      expected_output="Detailed Customer Issues mentioning Ticket Type, Product Purchased and NAME of Brand from '{brands_string}' report with trends and insights.",
      agent=analyst,
  )

task2 = Task(
      description="""Using the insights provided, develop an engaging post that highlights the top-customer issues for each of the brands in the dataset their pain points.
    Your post should be informative yet accessible, catering to a casual audience.For each of the Brand, categorize the insights based on information from TicketType, TickectSUbject and TicketDescription
    Make it sound cool, avoid complex words.""",
      expected_output="List in Bullet Points the customer issues. Ensure thet the post has Brand",
      agent=writer,
  )


In [164]:
crew = Crew(
      agents=[analyst,writer],
      tasks=[task1,task2],
      verbose=True,  # You can set it to 1 or 2 to different logging levels
  )

result = crew.kickoff()

# Agent: Customer Ticket Analyst
## Task: Analyze the top customer issues for each of the brands in '{brands_string}'.
      Use the columns Ticket Type, Ticket Subject, and Ticket Description to generate insights.


# Agent: Customer Ticket Analyst
## Thought: I need to analyze the top customer issues for each of the brands in the given string. To do this, I will use the Customer Support Query Tool to gather insights on customer issues based on Ticket Type, Ticket Subject, and Ticket Description.
## Using tool: Customer Support Query Tool
## Tool Input: 
"{\"query\": \"Analyze the top customer issues for each brand in {brands_string}. Focus on Ticket Type, Ticket Subject, and Ticket Description for trends and insights.\"}"
## Tool Output: 
To analyze the top customer issues for each brand, we can look at the ticket types, subjects, and descriptions for trends and insights:

1. **GoPro**: The main issue is a technical problem related to product setup, as indicated by a customer's reque

# Key Observations:

* The system automatically identifies brands from the dataset, ensuring accurate insights based on the available data.
* Duplicate entries are eliminated, so if a brand, such as Amazon, appears multiple times, the insights are consolidated into a single entry.
* From an initial dataset of 18 records, the final output consists of 12 unique insights.
* The process concludes with a brief summary of the findings, offering a concise overview of the key trends.

# Conclusion
As businesses continue to face increasing demands for quicker, more accurate support, adopting technologies like Agentic RAG can be a game-changer. It reduces the workload on human agents, allows teams to focus on more complex issues, and ensures that customers receive timely, relevant solutions. In a world where customer experience is key to business success, Agentic RAG’s blend of automation, efficiency, and intelligence is setting a new standard for customer service excellence.

By embracing Agentic RAG, companies can not only streamline their operations but also stay ahead in providing superior customer support in an ever-evolving digital landscape.